In [101]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, AppLayout, fixed, interact_manual
import ternary
# import mpltern
# import panel as pn
import pandas as pd
from matplotlib.patches import Wedge
%matplotlib inline
matplotlib.rcParams['figure.dpi'] = 150
matplotlib.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['font.sans-serif'] = ['DejaVu Serif']
import matplotlib.font_manager
from IPython.core.display import HTML
import matplotlib.path as mpath
star = mpath.Path.unit_regular_star(6)

# def make_html(fontname):
#     return "<p>{font}: <span style='font-family:{font}; font-size: 24px;'>{font}</p>".format(font=fontname)

# code = "\n".join([make_html(font) for font in sorted(set([f.name for f in matplotlib.font_manager.fontManager.ttflist]))])

# HTML("<div style='column-count: 2;'>{}</div>".format(code))


In [102]:
%%time
# output_file = 'pdfchem_output/output_11_10.dat'
# filename = 'output_sf'
filename = '_60_33'
output_file = f'output{filename}.dat'

# d = np.genfromtxt(output_file)

# #UV radiation (Draine); cosmic-ray ionization rate (s-1); metallicity (Zsolar)
# UV = d[:,0]; CR = d[:,1]; Z = d[:,2]
# #Gas temperature
# Tgas = d[:,3]
# #Abundances of CII; CI; CO; OI; HI; H2; OH+; H2O+; OH; CH; HCO+
# CII = d[:,14]; CI = d[:,28]; CO = d[:,31]
# OI = d[:,33]; HI = d[:,35]; H2 = d[:,34]
# OHp = d[:,12]; H2Op = d[:,16]; OH = d[:,32]
# CH = d[:,24]; HCOp = d[:,26]
# #
# # CII = d[:,14]; CI = d[:,28]; CO = d[:,31]
# tot_carbon = CII+CI+CO
# c2 = CII/tot_carbon
# c1 = CI/tot_carbon
# co = CO/tot_carbon

CPU times: total: 0 ns
Wall time: 0 ns


In [103]:
d = pd.read_table(output_file, header=None, sep='\s+')
d.rename(columns= {
         0: 'UV',
         1: 'CR',
         2: 'Z',
         3: 'Tgas',
         14: 'CII',
         28: 'CI',
         31: 'CO',
         33: 'OI',
         35: 'HI',
         34: 'H2',
         12: 'OHp',
         16: 'H2Op',
         32: 'OH',
         24: 'CH',
         26: 'HCOp',
         },
         inplace=True)
# allpoints = np.asarray(d[['CII','CI','CO']])

# CII = d['CII']; CI = d['CI']; CO = d['CO']
# tot_carbon = CII+CI+CO
# c2 = CII/tot_carbon
# c1 = CI/tot_carbon
# co = CO/tot_carbon
# allpoints = []
# for i in range(len(c1)):
#     allpoints.append((c2[i], c1[i], co[i]))

In [104]:
# uv =-0.8
# cr =-17
# uv_diff = np.abs(d['UV'] - 10**uv)
# cr_diff = np.abs(d['CR'] - 10**cr)
# matched_d = d[(np.abs(d['UV'] - 10**uv) == uv_diff.min()) & (np.abs(d['CR'] - 10**cr) == cr_diff.min())]
# CII = matched_d['CII']; CI = matched_d['CI']; CO = matched_d['CO']
# tot_carbon = CII+CI+CO
# c2 = CII/tot_carbon
# c1 = CI/tot_carbon
# co = CO/tot_carbon
# f"{c2[0]:.0%}", f"{c1[0]:.0%}", f"{co[0]:.0%}"
# pd.unique(d['UV'])[::5]

In [105]:
# make contours for UV and CR
for sub_cr in pd.unique(d['CR']):
    cr_g = d[d['CR'] == sub_cr]
    CII = cr_g['CII']; CI = cr_g['CI']; CO = cr_g['CO']
    tot_carbon = CII+CI+CO
    sub_c2,sub_c1,sub_co = CII/tot_carbon, CI/tot_carbon, CO/tot_carbon
    cr_points = []
    for i in range(len(sub_c1)):
        cr_points.append((sub_c2.iloc[i], sub_c1.iloc[i], sub_co.iloc[i]))
for sub_uv in pd.unique(d['UV']):
    uv_g = d[d['UV'] == sub_uv]
    CII = uv_g['CII']; CI = uv_g['CI']; CO = uv_g['CO']
    tot_carbon = CII+CI+CO
    sub_c2,sub_c1,sub_co = CII/tot_carbon, CI/tot_carbon, CO/tot_carbon
    uv_points = []
    for i in range(len(sub_c1)):
        uv_points.append((sub_c2.iloc[i], sub_c1.iloc[i], sub_co.iloc[i]))

In [106]:
def plot_carbon(uv, cr, z):
    # Create boolean masks for each condition
    uv_diff = np.abs(d['UV'] - 10**uv)
    cr_diff = np.abs(d['CR'] - 10**cr)
    # idx = np.where((uv_diff == uv_diff.min()) & (cr_diff == cr_diff.min()))[0]
    # uv_idx = np.where(uv_diff == uv_diff.min())[0]
    # cr_idx = np.where(cr_diff == cr_diff.min())[0]
    
    # Use the combined_mask to find the row(s) that satisfy all conditions
    matched_d = d[(np.abs(d['UV'] - 10**uv) == uv_diff.min()) & (np.abs(d['CR'] - 10**cr) == cr_diff.min())]
    CII = matched_d['CII']; CI = matched_d['CI']; CO = matched_d['CO']
    tot_carbon = CII+CI+CO
    c2 = CII/tot_carbon
    c1 = CI/tot_carbon
    co = CO/tot_carbon
    points = (c2.iloc[0], c1.iloc[0], co.iloc[0])
    color = (c2.iloc[0], c1.iloc[0], co.iloc[0], 1)
    scale = 1.0
    
    figure, tax = ternary.figure(scale=scale)
    figure.set_size_inches(5, 4.6)
    for sub_cr in pd.unique(d['CR'])[::2]:
        cr_g = d[d['CR'] == sub_cr]
        CII = cr_g['CII']; CI = cr_g['CI']; CO = cr_g['CO']
        tot_carbon = CII+CI+CO
        sub_c2,sub_c1,sub_co = CII/tot_carbon, CI/tot_carbon, CO/tot_carbon
        cr_points = []
        for i in range(len(sub_c1)):
            cr_points.append((sub_c2.iloc[i], sub_c1.iloc[i], sub_co.iloc[i]))
        tax.plot(cr_points, linewidth=2.0, c='xkcd:turquoise', alpha=0.3, label="Curve")
    for sub_uv in pd.unique(d['UV'])[::2]:
        uv_g = d[d['UV'] == sub_uv]
        CII = uv_g['CII']; CI = uv_g['CI']; CO = uv_g['CO']
        tot_carbon = CII+CI+CO
        sub_c2,sub_c1,sub_co = CII/tot_carbon, CI/tot_carbon, CO/tot_carbon
        uv_points = []
        for i in range(len(sub_c1)):
            uv_points.append((sub_c2.iloc[i], sub_c1.iloc[i], sub_co.iloc[i]))
        tax.plot(uv_points, linewidth=2.0, c='xkcd:indigo', alpha=0.4, label="Curve")

    # Draw Boundary and Gridlines
    tax.boundary(linewidth=2.0)
    tax.gridlines(color="silver", multiple=0.1)
    tax.ticks(clockwise=True, axis='lbr', ticks=['0%', '20%', '40%', '60%', '80%', '100%'], multiple=1, linewidth=2, offset=0.032)
    tax.get_axes().axis('off')
    tax.clear_matplotlib_ticks()

    # Set Axis labels and Title
    fontsize = 14
    offset = 0.14
    tax.right_corner_label("C$^+$", fontsize=fontsize, offset=0.1, c='crimson')
    tax.top_corner_label("CI", fontsize=fontsize, offset=0.25, c='forestgreen')
    tax.left_corner_label("CO", fontsize=fontsize, offset=0.05, c='steelblue')
    # tax.bottom_axis_label("CO $\\longleftarrow\\longleftarrow\\longleftarrow$", fontsize=fontsize, offset=offset, c='steelblue')
    # tax.right_axis_label("$\\longrightarrow\\longrightarrow\\longrightarrow$ C$^+$", fontsize=fontsize, offset=offset,  c='crimson')
    # tax.left_axis_label("CI $\\longrightarrow\\longrightarrow\\longrightarrow$", fontsize=fontsize, offset=offset, c='forestgreen')
    # tax.scatter(allpoints, alpha=0.2, marker='2', s=100)
    tax.scatter([points], c=color, s=200, marker=star, edgecolor='none', zorder=5)
    c2str, c1str, costr = f"{c2.iloc[0]:.0%}", f"{c1.iloc[0]:.0%}", f"{co.iloc[0]:.0%}"
    tax.set_title(f"C$^+$:{c2str}, CI:{c1str}, CO:{costr}\n\n", fontsize=fontsize,y=-0.03)
    tax.show()
    global filename
    figure.savefig(f"ternary{filename}.pdf")

In [107]:
uv_slider = widgets.FloatSlider(value=-0.4, min=-1, max=5, step=0.1, description='lg(UV):',layout=widgets.Layout(width='690px',font_size=16))
cr_slider = widgets.FloatSlider(value=-15.48, min=-17, max=-13, step=-0.1, description='lg(CR):',layout=widgets.Layout(width='690px'))
z_slider  = widgets.SelectionSlider(value=1, options=[0.1, 0.5, 1, 2], description='Metallicity:', layout=widgets.Layout(width='500px'))
sliders_box = widgets.VBox([uv_slider, cr_slider, #z_slider
                           ])
# myplot=widgets.interactive(plot_carbon, uv=uv_slider, cr=cr_slider, z=1.0)
myplot=widgets.interactive_output(plot_carbon, {'uv':uv_slider, 'cr':cr_slider, "z":fixed(1.0)})
# full_layout = widgets.VBox([sliders_box, myplot], layout=widgets.Layout(width='50%'))
AppLayout(footer=sliders_box,
          left_sidebar=None,
          center=myplot,
          right_sidebar=None,
          header=None)

AppLayout(children=(VBox(children=(FloatSlider(value=-0.4, description='lg(UV):', layout=Layout(width='690px')…